In [22]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os
import psycopg2
import datetime
import csv



In [65]:
PROCESSED_DATA_PATH = f'/Users/danielcollins/airflow/processed_data.csv'


In [66]:
PROCESSED_DATA_PATH

'/Users/danielcollins/airflow/processed_data.csv'

In [17]:
conn = psycopg2.connect(database="hockey", user="postgres", password="postgres", host="localhost", port="5432")

In [4]:
cur = conn.cursor()

In [68]:
df

,time,id
0,2021-11-28 16:05:24.792307,9863


In [67]:
df.to_sql('time_poos', con=conn, if_exists='append', index=False)

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': relation "sqlite_master" does not exist
LINE 1: SELECT name FROM sqlite_master WHERE type='table' AND name=?...
                         ^


In [15]:
try:
    cur.execute("""
    SELECT version()
    """
    )
    # print(cur.fetchall())
except Exception as e:
    print(e)

In [16]:
cur.fetchone()

('PostgreSQL 13.4 on x86_64-apple-darwin19.6.0, compiled by Apple clang version 11.0.3 (clang-1103.0.32.62), 64-bit',)

In [38]:
file = open()

2144

In [60]:
conn

<connection object at 0x7fced88f6970; dsn: 'user=postgres password=xxx dbname=hockey host=localhost port=5432', closed: 0>

In [73]:

print('transform_poopoo')
data = {}
time = datetime.datetime.now()
data['time'] = time
data['id'] = int(random.random()*10000)
data
df = pd.DataFrame([data])
# df = pd.DataFrame(data, index=data['id'])
df.to_csv(f'{os.getcwd()}/data_lol.csv',index=False, mode='a')

transform_poopoo


In [77]:
df.shape

(1, 2)

In [24]:
df.set_index('id', inplace=True)
df

transform_poopoo


In [71]:
df_new = pd.read_csv(f'{os.getcwd()}/data_lol.csv')

In [72]:
with open(f'{os.getcwd()}/data_lol.csv') as f:
    reader = csv.reader(f)
    a

,time,id
0,2021-11-28 16:05:24.792307,9863


In [69]:
os.getcwd()

'/Users/danielcollins/airflow/test_scripts'

In [59]:
df_new

,time
id,
9863,2021-11-28 16:05:24.792307


In [1]:
cf_url = 'https://www.capfriendly.com/teams/flames'
s_url = 'https://www.spotrac.com/nhl/calgary-flames/'

In [4]:
flames = pd.read_html(s_url)

In [5]:
print(type(flames))

<class 'list'>


In [16]:
flames[2]

,Center,Pos.,2021,2022,2023,2024,2025
0,Sean Monahan,C,"$6,375,000","$6,375,000",UFA,NaN,NaN
1,Mikael Backlund,C,"$5,350,000","$5,350,000",NaN,UFA,NaN
2,Blake Coleman,C,"$4,900,000","$4,900,000","$4,900,000","$4,900,000","$4,900,000"
3,Elias Lindholm,C,"$4,850,000","$4,850,000","$4,850,000",UFA,NaN
4,Dillon Dube,C,"$2,300,000","$2,300,000","$2,300,000",RFA,NaN
5,Adam Ruzicka,C,"$801,666",RFA,NaN,NaN,NaN
6,Trevor Lewis,C,"$800,000",UFA,NaN,NaN,NaN


In [10]:
for i in range(len(flames)):
    print(flames[i])

               Unnamed: 0         2021         2022         2023         2024  \
0   Total Cap Allocations  $82,198,416  $52,902,500  $32,050,000  $15,450,000   
1          Signed Players           26           13            8            3   
2                Forwards  $50,071,240  $29,852,500  $12,050,000   $4,900,000   
3              Defensemen  $22,750,000  $15,550,000  $14,000,000   $4,550,000   
4             Goaltending   $6,750,000   $6,750,000   $6,000,000   $6,000,000   
5         Injured Reserve   $2,596,750          NaN          NaN          NaN   
6               Suspended          NaN          NaN          NaN          NaN   
7  Buried/Buyout/Dead Cap   $1,500,000          NaN          NaN          NaN   

          2025  
0  $15,450,000  
1            3  
2   $4,900,000  
3   $4,550,000  
4   $6,000,000  
5          NaN  
6          NaN  
7          NaN  
           Left Wing Pos.        2021        2022 2023  2024  2025
0    Matthew Tkachuk   LW  $7,000,000         RFA 

In [15]:
flames[]

IndexError: list index out of range

In [17]:
flames_df = pd.DataFrame(flames[1])

In [30]:
flames_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"FORWARDS (13 - $49,351,666)","FORWARDS (13 - $49,351,666)",YEARS REMAINING,TERMS,POS,STATUS,ACQUIRED,AGE,CAP%,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27
1,"Tkachuk, Matthew","Tkachuk, Matthew",1 RFA,NaN,"LW, RW",NHL,Draft,23,8.6,"$7,000,000$7,000,000$9,000,000$9,000,000$0$0",RFA,NaN,NaN,NaN,NaN
2,"Gaudreau, Johnny","Gaudreau, Johnny",1 UFA,M-NTC,LW,NHL,Draft,28,8.3,"$6,750,000$6,750,000$6,750,000$6,750,000$0$0M-NTC",UFA,NaN,NaN,NaN,NaN
3,"Monahan, Sean ""A""","Monahan, Sean ""A""",2 UFA,M-NTC,C,NHL,Draft,27,7.8,"$6,375,000$6,375,000$6,125,000$6,125,000$0$0M-NTC","$6,375,000$6,375,000$6,000,000$6,000,000$0$0M-NTC",UFA,NaN,NaN,NaN
4,"Backlund, Mikael ""A""","Backlund, Mikael ""A""",3 UFA,M-NTC,C,NHL,Draft,32,6.6,"$5,350,000$5,350,000$5,350,000$5,350,000$0$0M-NTC","$5,350,000$5,350,000$5,350,000$5,350,000$0$0M-NTC","$5,350,000$5,350,000$5,350,000$5,350,000$0$0M-NTC",UFA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,GOALIES (2),GOALIES (2),YEARS REMAINING,TERMS,POS,STATUS,ACQUIRED,AGE,CAP%,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27
96,"Wolf, Dustin","Wolf, Dustin",3 RFA,ELC,G,Minor,Draft,20,-,"$813,333$825,000$70,000$70,000$20,000$82,500","$813,333$825,000$70,000$70,000$0$85,000","$813,333$825,000$70,000$70,000$15,000$0",RFA,NaN,NaN
97,"Werner, Adam","Werner, Adam",1 RFA,NaN,G,Minor,Signed,24,-,"$750,000$750,000$100,000$100,000$0$0",RFA,NaN,NaN,NaN,NaN
98,TOTAL,TOTAL,NaN,NaN,NaN,NaN,NaN,22.0,-,"$1,563,333$1,575,000$170,000$170,000$20,000$82...","$813,333$825,000$70,000$70,000$0$85,000","$813,333$825,000$70,000$70,000$15,000$0",NaN,NaN,NaN
